# Implement VAR to yahoo finance Week 2

In [1]:
import pandas as pd
import numpy as np

## Plotting
import matplotlib.pyplot as plt
import seaborn as sns

from datetime import datetime
from sklearn.metrics import mean_squared_error

from statsmodels.tsa.api import VAR
from statsmodels.tsa.stattools import acf, pacf, grangercausalitytests
from statsmodels.tsa.statespace.varmax import VARMAX

## Import TimeSeriesSplit
from sklearn.model_selection import TimeSeriesSplit
from sklearn.preprocessing import StandardScaler

import yfinance as yf

from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.stattools import kpss

## def of prediction_class

In [2]:
def prediction_class(df):
    df['Prediction'] = np.where(df['Close'] < df['Close'].shift(-1), 1, 0)
    return df

## Import from Yahoo Finance

In [3]:
Ford = yf.download('F', period="3mo")

[*********************100%***********************]  1 of 1 completed


In [4]:
Ford['change_in_price'] = Ford['Close'].diff()
Ford

,Open,High,Low,Close,Adj Close,Volume,change_in_price
Date,,,,,,,
2024-08-16,10.40,10.52,10.37,10.51,10.369115,34142600,NaN
2024-08-19,10.56,10.75,10.54,10.72,10.576301,41066300,0.210000
2024-08-20,10.70,10.79,10.61,10.68,10.536837,32468000,-0.040000
2024-08-21,10.84,10.97,10.75,10.85,10.704557,48943900,0.170000
2024-08-22,10.85,11.01,10.84,10.92,10.773619,50885400,0.070000
...,...,...,...,...,...,...,...
2024-11-11,11.07,11.32,11.02,11.23,11.230000,56630500,0.259999
2024-11-12,11.14,11.30,10.95,11.10,11.100000,44921400,-0.129999
2024-11-13,11.12,11.17,11.02,11.10,11.100000,55150900,0.000000


In [5]:
Toyota = yf.download('TM', period="3mo")
Toyota['change_in_price'] = Toyota['Close'].diff()

[*********************100%***********************]  1 of 1 completed


## Split the data

In [6]:
#df_F_test = Ford[-5:].copy()
#df_F_test

In [7]:
df = pd.concat([Ford['change_in_price'], Toyota['change_in_price']], axis = 1)

In [8]:
df.columns = ['F_change_in_price', 'TM_change_in_price']

In [9]:
df

,F_change_in_price,TM_change_in_price
Date,,
2024-08-16,NaN,NaN
2024-08-19,0.210000,1.029999
2024-08-20,-0.040000,-2.360001
2024-08-21,0.170000,2.279999
2024-08-22,0.070000,-2.119995
...,...,...
2024-11-11,0.259999,2.070007
2024-11-12,-0.129999,0.889999
2024-11-13,0.000000,-2.639999


## Apply VAR model

In [10]:
model = VAR(df.dropna())
results = model.fit(22)
predictions = results.forecast(df.values[-22:], steps=5)[:, 0]

C:\Users\teres\anaconda3\envs\equity_vs_commodity\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)


In [11]:
predictions

array([-0.76014287,  0.26254038,  1.06435679,  0.0798689 , -1.21950835])

In [12]:
#df_F_test

In [13]:
#accuracy_TM = sum(predictions*df_F_test['change_in_price'] > 0)/5

In [14]:
#accuracy_TM